In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import warnings

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings(action='ignore') 

from src import crs, PublicPredictor
from src.dbc import utils
from IPython.display import clear_output

In [3]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)

# 3. data init
_month = 1

month_df = pd.DataFrame(m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

PUBLIC_PERCENTAGE = 30
APT = crs.utils.get_APT(month_df, PUBLIC_PERCENTAGE)

calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
apt = calc.apart

public_kwh = APT - month_df['usage (kWh)'].sum()
public_kwh

13330

In [10]:
from src import SavingFeedback

# 1. Data Pre-processing
ksf = SavingFeedback(xlsx)
ksf.select_month(1)

# 2. KMeans Run
ksf.kmeans_run()

# 3. adjust anomaly
ksf.adjust_anomaly()
anomaly = ksf.check_anomaly()
print("이상패턴 조정 확인", len(anomaly) == 0)

time_size = 3

ksf.time_based_grouping(time_size)
ksf.feedback()
ksf.result()

kmeans_time_group = ksf.new_group.copy()
kmeans_time_group.head()

ECV : 51 %
이상패턴 조정 확인 True


,name,usage (kWh),label
0,아파트1-104-1206,99,0
1,아파트1-104-303,10,0
2,아파트1-104-1307,23,0
3,아파트1-104-1208,40,0
4,아파트1-104-408,39,0


In [30]:
round(kmeans_time_group['usage (kWh)'].mean())

231

In [13]:
tsf = SavingFeedback(xlsx, _type="tdkmeans")

tsf.select_month(1)
tsf.kmeans_run()

tsf.adjust_anomaly()
anomaly = tsf.check_anomaly()
print("이상패턴 조정 확인", len(anomaly) == 0)

time_size = 3

tsf.time_based_grouping(time_size)
tsf.feedback()
tsf.result()

tdkmeans_time_group = tsf.new_group.copy()


clear_output(wait=True)
tdkmeans_time_group.head()

,name,usage (kWh),label
0,아파트1-104-1206,99,0
1,아파트1-104-303,10,0
2,아파트1-104-1307,23,0
3,아파트1-104-1208,40,0
4,아파트1-104-408,39,0


In [16]:
round(tdkmeans_time_group['usage (kWh)'].mean())

241

In [32]:
kmeans_APT = round(kmeans_time_group['usage (kWh)'].sum()) + public_kwh
tdkmeans_APT = round(tdkmeans_time_group['usage (kWh)'].sum()) + public_kwh

print(kmeans_APT, tdkmeans_APT)

38296 39307


In [36]:
kmeans_calc = crs.models.ManagementOffice(
    month=_month,
    households=kmeans_time_group.copy(),
    APT=kmeans_APT,
    contract='단일계약'
)
tdkmeans_calc = crs.models.ManagementOffice(
    month=_month,
    households=tdkmeans_time_group.copy(),
    APT=tdkmeans_APT,
    contract='단일계약'
)

In [45]:
# APT 평균 사용량
print(calc.apart.kwh, kmeans_calc.apart.kwh, tdkmeans_calc.apart.kwh)

# 세대부 평균 사용량
print(round(month_df['usage (kWh)'].mean()),
     round(kmeans_time_group['usage (kWh)'].mean()),
     round(tdkmeans_time_group['usage (kWh)'].mean()))


# 아파트 전체요금
print(calc.bill, kmeans_calc.bill, tdkmeans_calc.bill)

# 세대부 전체요금
print(calc.bill - calc.public_bill,
     kmeans_calc.bill - kmeans_calc.public_bill,
     tdkmeans_calc.bill - tdkmeans_calc.public_bill)

# 공용부 요금
print(calc.public_bill,
     kmeans_calc.public_bill,
     tdkmeans_calc.public_bill)

411 355 364
288 231 241
6439540 4763500 4923370
3798980 2637760 2871750
2640560 2125740 2051620
